In [2]:
import joblib
import pandas as pd
import numpy as np
import glob

import pandas as pd
import numpy as np

In [3]:
lista_itens_csv = glob.glob('../resources/itens/itens/*.csv')

df_itens_completo = pd.concat([pd.read_csv(file) for file in lista_itens_csv], ignore_index=True)

In [4]:
df_train = pd.read_csv("../resources/files/treino/treino_parte1.csv")

In [5]:
# import models
vectorizer = joblib.load('vectorizer.pkl')
kmeans = joblib.load('kmeans.pkl')

In [6]:
#### tratamento df_train V3
def combine_text(df):
    return df["title"] + " " + df["caption"] + " " + df["body"]


def limpar_valor(valor):
    valor_limpo = (valor.
                replace('\n', ' ').
                replace("'", ' ').
                replace("[", ' ').
                replace("]", ' ').
                replace(',', ' ').
                strip().split()
        ) 
    
    return valor_limpo


def tratar_base_treino(df, df_itens, model, vectorizer):
    df = df.set_index("userId").to_dict(orient="index")
    df_users = pd.DataFrame()
    n_iteractions = 0
    total_iteractions = len(df)
    df_itens_indexed = df_itens.set_index("page").to_dict(orient="index")
    for key in df:
        row = df[key]
        user = key
        print(f'user: {user}')
        hist = limpar_valor(row['history'])
        print(f'hist: {hist}')
        
        timestampHistory = list(map(int, limpar_valor(row['timestampHistory'])))
        print(f'timestampHistory: {timestampHistory}')
        
        numberOfClicksHistory = list(map(int, limpar_valor(row['numberOfClicksHistory'])))
        print(f'numberOfClicksHistory: {numberOfClicksHistory}')
        
        timeOnPageHistory = list(map(int, limpar_valor(row['timeOnPageHistory'])))
        print(f'timeOnPageHistory: {timeOnPageHistory}')
        
        scrollPercentageHistory = list(map(float, limpar_valor(row['scrollPercentageHistory'])))
        print(f'scrollPercentageHistory: {scrollPercentageHistory}')
        
        pageVisitsCountHistory = list(map(int, limpar_valor(row['pageVisitsCountHistory'])))
        print(f'pageVisitsCountHistory: {pageVisitsCountHistory}')
        
        timestampHistory_max = max(timestampHistory)
        
        numberOfClicksHistory_mean = np.mean(numberOfClicksHistory)
        numberOfClicksHistory_std = max(np.std(numberOfClicksHistory), 1)
        
        timeOnPageHistory_mean = np.mean(timeOnPageHistory)
        timeOnPageHistory_std = max(np.std(timeOnPageHistory), 1)
        
        scrollPercentageHistory_mean = np.mean(scrollPercentageHistory)
        scrollPercentageHistory_std = max(np.std(scrollPercentageHistory), 1)
        
        pageVisitsCountHistory_mean = np.mean(pageVisitsCountHistory)
        pageVisitsCountHistory_std = max(np.std(pageVisitsCountHistory), 1)
        
        user_dict = {'userId': user, **{f'{i}': 0 for i in model.labels_}}
        for n_item in range(len(hist)):
            timestampHistory_score = timestampHistory[n_item] / timestampHistory_max
            numberOfClicksHistory_score = (numberOfClicksHistory[n_item] - numberOfClicksHistory_mean) / numberOfClicksHistory_std
            timeOnPageHistory_score = (timeOnPageHistory[n_item] - timeOnPageHistory_mean) / timeOnPageHistory_std
            scrollPercentageHistory_score = (scrollPercentageHistory[n_item] - scrollPercentageHistory_mean) / scrollPercentageHistory_std
            pageVisitsCountHistory_score = (pageVisitsCountHistory[n_item] - pageVisitsCountHistory_mean) / pageVisitsCountHistory_std
            
            final_item_score = timestampHistory_score * 1.3 + numberOfClicksHistory_score * 1 + timeOnPageHistory_score * 1 + scrollPercentageHistory_score * 1.2 + pageVisitsCountHistory_score * 1
            item_row = df_itens_indexed.get(hist[n_item], None)
            if len(item_row) == 0 or item_row == None:
                print(" --- Item não encontrado na base de dados --- ")
                continue
            X = vectorizer.transform([combine_text(item_row)])
            cluster = model.predict(X)[0]
            
            user_dict[f'{cluster}'] += final_item_score
            
        df_users = pd.concat([df_users, pd.DataFrame([user_dict])], ignore_index=True)
        
        # Porcentagem de progresso
        n_iteractions += 1        
        progresso = (n_iteractions / total_iteractions) * 100
        print(f"Processando: {progresso:.2f}% ", end="\r")
        
    return df_users, hist


def dict_recomendations(df_user):
    sum = df_user.iloc[0, 1:].sum()
    n_recomendations = 10
    user_dict = {f'{df_user.columns[n]}': round((df_user[df_user.columns[n]].values[0] / sum) * n_recomendations) for n in range(len(df_user.columns)) if df_user.columns[n] != "userId"}
    return dict(sorted(user_dict.items(), key=lambda item: item[1], reverse=True))


def recomend(user_dict, user_hist, last_news, n_recomendations = 10):
    n = 0
    n_recomendations = n_recomendations
    df = pd.DataFrame()
    for key in user_dict:
        value = user_dict[key]
        if value <= 0:
            continue
        
        if value >= n_recomendations:
            value = n_recomendations
            
        
        if n + value <= n_recomendations:
            n += value
            temp = last_news[last_news['cluster'] == int(key)].head(value)
            df = pd.concat([df, temp], ignore_index=True)        
            
            continue
            
        
        diff = n_recomendations - n
        
        if diff > 0:
            first_key = next(iter(user_dict))
            temp = last_news[last_news['cluster'] == int(first_key)].head(diff)
            df = pd.concat([df, temp], ignore_index=True)
        
    if len(df) < n_recomendations:
        df = pd.concat([df, last_news[~last_news['page'].isin(user_hist)].sort_values(by='issued').head(n_recomendations)])
    return df.head(n_recomendations)


    

In [7]:
last_news = pd.read_csv('last_news.csv')
X = vectorizer.transform(combine_text(last_news))
last_news['cluster'] = kmeans.predict(X)

In [8]:
df_user, user_hist = tratar_base_treino(df=df_train.sample(1), df_itens=df_itens_completo, model=kmeans, vectorizer=vectorizer)

user: d22280aa4221e3960866ac9dc023921c25a25652fc8ef0c07813a9e874a71eec
hist: ['2c196b72-302b-4120-ac00-2498b8021dd5']
timestampHistory: [1658976513760]
numberOfClicksHistory: [11]
timeOnPageHistory: [20500]
scrollPercentageHistory: [29.71]
pageVisitsCountHistory: [1]


In [14]:
df_user

,userId,1,6,15,10,12,2,11,19,3,...,18,4,7,9,0,16,14,5,17,8
0,d22280aa4221e3960866ac9dc023921c25a25652fc8ef0...,0,1.3,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [11]:
user_dict = dict_recomendations(df_user=df_user)
user_dict

{'6': 10,
 '1': 0,
 '15': 0,
 '10': 0,
 '12': 0,
 '2': 0,
 '11': 0,
 '19': 0,
 '3': 0,
 '13': 0,
 '18': 0,
 '4': 0,
 '7': 0,
 '9': 0,
 '0': 0,
 '16': 0,
 '14': 0,
 '5': 0,
 '17': 0,
 '8': 0}

In [13]:
rec = recomend(user_dict=user_dict, user_hist=user_hist, last_news=last_news, n_recomendations=10)
rec

,page,url,issued,modified,title,body,caption,cluster
0,ebd223ab-9d97-429f-bf02-1781477f2653,http://g1.globo.com/fantastico/noticia/2022/08...,2022-08-15 02:57:16+00:00,2022-08-15 02:57:17+00:00,Varíola dos macacos: é possível conter o surto...,Vacina contra varíola pode chegar em setembro ...,"O governo anunciou que chegam ao Brasil, em se...",6
1,340f9f0b-98b1-4a18-a339-7bf927198911,http://g1.globo.com/fantastico/noticia/2022/08...,2022-08-15 02:40:07+00:00,2022-08-15 02:40:07+00:00,Brad Pitt diz que teve dificuldade em cenas de...,Brad Pitt é dirigido por ex-dublê em comédia c...,Longa reúne estrelas consagradas como Sandra B...,6
2,292d82a1-5e96-4b3b-8ee7-e696f3aa771c,http://g1.globo.com/fantastico/noticia/2022/08...,2022-08-15 02:31:41+00:00,2022-08-15 02:31:41+00:00,'Bichos na Escuta - Salvei o meu cachorro': co...,'Bichos na Escuta - Salvei o meu cachorro': co...,No primeiro episódio de “Bichos na Escuta – Sa...,6
3,78cd9782-3466-4808-a231-41608c2e4d1b,http://g1.globo.com/fantastico/noticia/2022/08...,2022-08-15 02:24:26+00:00,2022-08-15 04:24:25+00:00,Lucy Alves teve ajuda de Thaynara OG para trab...,Lucy Alves teve ajuda de Thaynara OG para trab...,"Como a atriz é de João Pessoa, teve que trabal...",6
4,c2fe4ec2-0a35-437b-86f3-f1ce1057fa68,http://g1.globo.com/fantastico/noticia/2022/08...,2022-08-15 02:21:37+00:00,2022-08-15 09:51:50+00:00,"Salman Rushdie, alvo de ataque em Nova York, p...","Salman Rushdie, alvo de ataque em Nova York, p...","Há 30 anos, o escritor foi marcado para morrer...",6
5,9c764c3a-f9f8-4fb2-b2c4-6331eaeb3dd6,http://g1.globo.com/fantastico/noticia/2022/08...,2022-08-15 02:04:27+00:00,2022-08-15 10:59:19+00:00,Áudios e vídeos revelam como os criminosos mai...,Áudios e vídeos revelam como os criminosos mai...,Polícia Federal frustrou o plano de fuga de Ma...,6
6,40908b83-0102-4e6c-9467-58dd76658be0,http://g1.globo.com/fantastico/noticia/2022/08...,2022-08-15 01:46:09+00:00,2022-08-15 02:01:47+00:00,\nInvestigação revela como uma rede de aviões ...,O Fantástico investiga a ação de aeronaves do ...,"Durante três dias, Sônia Bridi e Paulo Zero ac...",6
7,9fd6edda-5f28-4d58-a726-89da76b2a948,http://g1.globo.com/fantastico/noticia/2022/08...,2022-08-15 01:28:22+00:00,2022-08-15 01:28:23+00:00,Celebração em família é presente de Dia dos Pa...,Dia dos Pais: Djavan fala sobre o novo clipe c...,O cantor recebeu Poliana Abritta em um orquidá...,6
8,eb23272d-8e6c-479d-b972-eabeb5f6f3dd,http://g1.globo.com/fantastico/noticia/2022/08...,2022-08-15 01:06:34+00:00,2022-08-15 01:06:36+00:00,"'Ela se tornou minha inimiga', diz idosa vítim...",Operação prende filha que extorquiu dinheiro d...,"Genevieve Boghici, de 83 anos, foi mantida em ...",6
9,3ff02906-179b-4d74-a455-07c0663f7d08,http://g1.globo.com/fantastico/noticia/2022/08...,2022-08-15 00:45:38+00:00,2022-08-15 00:45:39+00:00,Criança Esperança: projeto Jovens do Futuro of...,Projeto apoiado pelo Criança Esperança em Salv...,Outra iniciativa deste mesmo projeto é a horta...,6


In [ ]:
rec.to_dict(orient="index")

{0: {'page': 'ebd223ab-9d97-429f-bf02-1781477f2653',
  'url': 'http://g1.globo.com/fantastico/noticia/2022/08/14/variola-dos-macacos-e-possivel-conter-o-surto-ou-a-doenca-vai-se-alastrar-ainda-mais-cientistas-opinam.ghtml',
  'issued': '2022-08-15 02:57:16+00:00',
  'modified': '2022-08-15 02:57:17+00:00',
  'title': 'Varíola dos macacos: é possível conter o surto ou a doença vai se alastrar ainda mais? Cientistas opinam',
  'body': 'Vacina contra varíola pode chegar em setembro no Brasil\nEsta semana, em São José do Rio Preto, interior de São Paulo, pelo menos 11 macacos foram envenenados ou agredidos e oito morreram: uma consequência grave de informações falsas que circulam sobre a varíola dos macacos.\n"Os primatas que a gente tem aqui no Brasil - o bugio, o macaco-prego, o sagui, os micos - não são reservatórios da varíola dos macacos. Eles não transmitem a doença para humanos”, alertou nas redes sociais um veterinário do Paraná.\nA doença só é chamada de varíola dos macacos porque

: 